# Creating, Sending and Receiving Packets

In [2]:
from scapy.all import *

In [2]:
conf.verb=1
conf.color_theme = RastaTheme()

## Stacking Layers

- the `/` operator is used as a composition operator between two layers
- the lower layer can have one or more of its defaults fields overloaded according to the upper layer
    - you still can give the value you want
    - a string can be used as a raw layer
    
<img src="http://scapy.readthedocs.io/en/latest/_images/fieldsmanagement.png" />

In [3]:
IP()

<IP  |>

In [4]:
IP()/TCP()

<IP  frag=0 proto=tcp |<TCP  |>>

In [5]:
Ether()/IP()/TCP()

<Ether  type=IPv4 |<IP  frag=0 proto=tcp |<TCP  |>>>

In [6]:
IP()/TCP()/"GET / HTTP/1.0\r\n\r\n"

<IP  frag=0 proto=tcp |<TCP  |<Raw  load='GET / HTTP/1.0\r\n\r\n' |>>>

In [7]:
Ether()/IP()/IP()/UDP()

<Ether  type=IPv4 |<IP  frag=0 proto=ipencap |<IP  frag=0 proto=udp |<UDP  |>>>>

In [9]:
IP(proto=55)/TCP(dport=-22)

<IP  frag=0 proto=55 |<TCP  dport=-22 |>>

## Build ARP packets
- build ARP packet which has two layers - Ether and ARP

In [10]:
ls(Ether)

dst        : DestMACField                        = (None)
src        : SourceMACField                      = (None)
type       : XShortEnumField                     = (36864)


In [11]:
ls(ARP)

hwtype     : XShortField                         = (1)
ptype      : XShortEnumField                     = (2048)
hwlen      : FieldLenField                       = (None)
plen       : FieldLenField                       = (None)
op         : ShortEnumField                      = (1)
hwsrc      : MultipleTypeField                   = (None)
psrc       : MultipleTypeField                   = (None)
hwdst      : MultipleTypeField                   = (None)
pdst       : MultipleTypeField                   = (None)


In [60]:
arpPkt = Ether()/ARP()

In [61]:
arpPkt.show()

###[ Ethernet ]### 
  dst= 00:50:56:e4:5b:86
  src= 00:0c:29:17:1a:37
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 00:0c:29:17:1a:37
     psrc= 192.168.195.161
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [62]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'

In [17]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 00:0c:29:17:1a:37
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 00:0c:29:17:1a:37
     psrc= 192.168.195.161
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [63]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'

In [64]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 00:0c:29:17:1a:37
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 00:0c:29:17:1a:37
     psrc= 192.168.195.161
     hwdst= FF:FF:FF:FF:FF:FF
     pdst= 0.0.0.0



In [65]:
# change dest ip
arpPkt['ARP'].pdst = '192.168.195.167'

In [70]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 00:0c:29:17:1a:37
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 00:0c:29:17:1a:37
     psrc= 192.168.195.161
     hwdst= FF:FF:FF:FF:FF:FF
     pdst= 192.168.195.167



## Capture packet using wireshark
- ARP packet is ready to be sent
- Open Wireshark and start capturing traffic

# Send packets - one way
- sendp() - works at layer two; can send ARP, Ethernet, etc.
- send() - send packets at layer 3 (IP, TCP, HTTP, etc.); will handle routing and layer 2
    - if return_packets=True is passed as parameter, these functions will also return received packes

In [67]:
help(send)

Help on function send in module scapy.sendrecv:

send(x, inter=0, loop=0, count=None, verbose=None, realtime=None, return_packets=False, socket=None, iface=None, *args, **kargs)
    Send packets at layer 3
    
    :param x: the packets
    :param inter: time (in s) between two packets (default 0)
    :param loop: send packet indefinetly (default 0)
    :param count: number of packets to send (default None=1)
    :param verbose: verbose mode (default None=conf.verbose)
    :param realtime: check that a packet was sent before sending the next one
    :param return_packets: return the sent packets
    :param socket: the socket to use (default is conf.L3socket(kargs))
    :param iface: the interface to send the packets on
    :param monitor: (not on linux) send in monitor mode
    :returns: None



In [68]:
help(sendp)

Help on function sendp in module scapy.sendrecv:

sendp(x, inter=0, loop=0, iface=None, iface_hint=None, count=None, verbose=None, realtime=None, return_packets=False, socket=None, *args, **kargs)
    Send packets at layer 2
    
    :param x: the packets
    :param inter: time (in s) between two packets (default 0)
    :param loop: send packet indefinetly (default 0)
    :param count: number of packets to send (default None=1)
    :param verbose: verbose mode (default None=conf.verbose)
    :param realtime: check that a packet was sent before sending the next one
    :param return_packets: return the sent packets
    :param socket: the socket to use (default is conf.L3socket(kargs))
    :param iface: the interface to send the packets on
    :param monitor: (not on linux) send in monitor mode
    :returns: None



In [71]:
# send the arpPkt - use sendp() because ARP is layer 2 packet
sendp(arpPkt)


Sent 1 packets..


In [72]:
response = srp1(arpPkt)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [73]:
response.show()

###[ Ethernet ]### 
  dst= 00:0c:29:17:1a:37
  src= 00:0c:29:f5:94:92
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= 6
     plen= 4
     op= is-at
     hwsrc= 00:0c:29:f5:94:92
     psrc= 192.168.195.167
     hwdst= 00:0c:29:17:1a:37
     pdst= 192.168.195.161
###[ Padding ]### 
        load= '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'



In [74]:
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.195.171                  (incomplete)                              eth0
192.168.195.254          ether   00:50:56:ff:e7:19   C                     eth0
192.168.195.2            ether   00:50:56:e4:5b:86   C                     eth0


In [36]:
# one liner
sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="00:11:22:aa:bb:cc")/ARP(hwsrc="00:11:22:aa:bb:cc",pdst="192.168.231.2"))


Sent 1 packets..


## Sending HTTP request

In [75]:
httpPkt = IP(dst="example.com")/TCP(dport=443)/"GET /index.html HTTP/1.1\r\n\r\n"

In [76]:
bytes(httpPkt)

b'E\x00\x00D\x00\x01\x00\x00@\x06\xc0\x8e\xc0\xa8\xc3\xa1]\xb8\xd8"\x00\x14\x01\xbb\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\x12\x16\x00\x00GET /index.html HTTP/1.1\r\n\r\n'

In [77]:
hexdump(httpPkt)

0000  45 00 00 44 00 01 00 00 40 06 C0 8E C0 A8 C3 A1  E..D....@.......
0010  5D B8 D8 22 00 14 01 BB 00 00 00 00 00 00 00 00  ].."............
0020  50 02 20 00 12 16 00 00 47 45 54 20 2F 69 6E 64  P. .....GET /ind
0030  65 78 2E 68 74 6D 6C 20 48 54 54 50 2F 31 2E 31  ex.html HTTP/1.1
0040  0D 0A 0D 0A                                      ....


In [78]:
httpPkt.show()

###[ IP ]### 
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= tcp
  chksum= None
  src= 192.168.195.161
  dst= Net('example.com')
  \options\
###[ TCP ]### 
     sport= ftp_data
     dport= https
     seq= 0
     ack= 0
     dataofs= None
     reserved= 0
     flags= S
     window= 8192
     chksum= None
     urgptr= 0
     options= []
###[ Raw ]### 
        load= 'GET /index.html HTTP/1.1\r\n\r\n'



## Send and Receive Packets
- Send and receive functions are the heart of Scapy
- sr() - send L3 packets and receive answers; returns answered and unanswered packets
- sr1() - returns only one packet that answered the sent packet
- sr() and sr1() functions can only send layer 3 L3 packets (IP, HTTP, etc.)
- srp() - send and receive L2 packets (ARP, Ehternet, 802.3, etc)
- srp1() - send and receive one layer 2 packet

In [79]:
response = sr(httpPkt)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


In [80]:
response

(<Results: TCP:1 UDP:0 ICMP:0 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [83]:
response[1].show()

In [84]:
response[1].summary()

## TCP/IP full three-way handshake

### send SYN packet

In [85]:
# Create syn packet
syn = IP(dst='example.com')/TCP(dport=443, flags='S')

In [86]:
syn.show()

###[ IP ]### 
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= tcp
  chksum= None
  src= 192.168.195.161
  dst= Net('example.com')
  \options\
###[ TCP ]### 
     sport= ftp_data
     dport= https
     seq= 0
     ack= 0
     dataofs= None
     reserved= 0
     flags= S
     window= 8192
     chksum= None
     urgptr= 0
     options= []



In [87]:
# Receive SYN-ACK packet
syn_ack = sr1(syn)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


In [88]:
syn_ack.show()

###[ IP ]### 
  version= 4
  ihl= 5
  tos= 0x0
  len= 44
  id= 2061
  flags= 
  frag= 0
  ttl= 128
  proto= tcp
  chksum= 0x789a
  src= 93.184.216.34
  dst= 192.168.195.161
  \options\
###[ TCP ]### 
     sport= https
     dport= ftp_data
     seq= 656966020
     ack= 1
     dataofs= 6
     reserved= 0
     flags= SA
     window= 64240
     chksum= 0x3884
     urgptr= 0
     options= [('MSS', 1460)]
###[ Padding ]### 
        load= '\x00\x00'



In [89]:
# Send Get request
getStr = 'GET / HTTP1.1\r\nHost:example.com\r\n\r\n'
request = IP(dst='example.com') / TCP(dport=443, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr
reply = sr1(request)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


In [90]:
reply.show()

###[ IP ]### 
  version= 4
  ihl= 5
  tos= 0x0
  len= 40
  id= 2082
  flags= 
  frag= 0
  ttl= 128
  proto= tcp
  chksum= 0x7889
  src= 93.184.216.34
  dst= 192.168.195.161
  \options\
###[ TCP ]### 
     sport= https
     dport= ftp_data
     seq= 656966021
     ack= 0
     dataofs= 5
     reserved= 0
     flags= R
     window= 32767
     chksum= 0xcb3f
     urgptr= 0
     options= []
###[ Padding ]### 
        load= '\x00\x00\x00\x00\x00\x00'



In [52]:
reply.summary()

'IP / TCP 93.184.216.34:http > 192.168.195.161:ftp_data R / Padding'

In [53]:
tcpip = IP(dst="example.com") / TCP(dport=80) # tcp packet needs dest port
reply = sr(tcpip)
reply

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


(<Results: TCP:1 UDP:0 ICMP:0 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [54]:
ans, unans = reply 

In [55]:
ans.summary()

IP / TCP 192.168.195.161:ftp_data > 93.184.216.34:http S ==> IP / TCP 93.184.216.34:http > 192.168.195.161:ftp_data SA / Padding


In [56]:
unans

<Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>

## TCP Syn Flooding
-  Not quite working with Scapy but should world with netwox
- form of DoS attack where attacker doesn't complete 3-way handshake by not sending ACK packet after receiving SYN-ACK from server
- attacker typically floods the victim's half-opened connections
- when queue is full, victim cannot take any more new connection
- See Taks 1 here: http://www.cis.syr.edu/~wedu/seed/Labs_16.04/Networking/TCP_Attacks/TCP_Attacks.pdf
- queue size can be viwed by using the following command

In [58]:
# on a server VM, run the following command
# Run SEEDLab VM which has Apache server running
! sysctl -q net.ipv4.tcp_max_syn_backlog

net.ipv4.tcp_max_syn_backlog = 512


In [59]:
# Turn of SYN cookie countermeasure
! sudo sysctl -a | grep cookie #(Display the SYN cookie flag)

net.ipv4.tcp_syncookies = 1
net.netfilter.nf_conntrack_sctp_timeout_cookie_echoed = 3
net.netfilter.nf_conntrack_sctp_timeout_cookie_wait = 3


In [ ]:
! sysctl -w net.ipv4.tcp_syncookies=0 #(turn off SYN cookie)

In [ ]:
# the following command shows all the halfopen connections
! netstat -na | grep SYN-RECV

In [ ]:
# send SYN PACKET To the server and check for SYN-RECV status
pkt = IP(dst="192.168.100.5")/TCP(dport=80)
reply = sr1(pkt)

In [ ]:
reply.show()

## Create ping request
- IP packet with ICMP echo request and some payload
- send the packet and capture it with wireshark

In [3]:
ping = IP(src="192.168.195.161", dst="192.168.195.167") / ICMP() / "Hey there, are you up?"

In [4]:
ping.show()

###[ IP ]### 
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = None
  src       = 192.168.195.161
  dst       = 192.168.195.167
  \options   \
###[ ICMP ]### 
     type      = echo-request
     code      = 0
     chksum    = None
     id        = 0x0
     seq       = 0x0
###[ Raw ]### 
        load      = 'Hey there, are you up?'



In [5]:
send(ping)


Sent 1 packets.
.

In [6]:
reply = sr1(ping)
reply.show()

Begin emission:
Finished sending 1 packets.
*
Received 1 packets, got 1 answers, remaining 0 packets
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 50
  id        = 30319
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = icmp
  chksum    = 0xfbc1
  src       = 192.168.195.167
  dst       = 192.168.195.161
  \options   \
###[ ICMP ]### 
     type      = echo-reply
     code      = 0
     chksum    = 0x4c09
     id        = 0x0
     seq       = 0x0
###[ Raw ]### 
        load      = 'Hey there, are you up?'



In [ ]:
reply['Raw']

## Exercise
- Spoof src IP and ping some IP in research/controlled network
- 